In [1]:
import datetime
import os
import random
import xarray as xr
import metpy
import cartopy.crs as ccrs
from pyresample import geometry, grid
import numpy as np
import matplotlib.pyplot as plt
import netCDF4

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyresample/bilinear/__init__.py:49: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


Define parameters

In [2]:
pc = ccrs.PlateCarree() #equidistant cylindrical projection
pc_extents = [-124.25, -114.25, 32.25, 42.25] #lons, lats
pc_params = pc.proj4_params #projection parameters loaded from proj4
target_extents = [pc_extents[0], pc_extents[2], pc_extents[1], pc_extents[3]] #reorder the lons and lats
target_rows, target_cols = 500, 500
shape = [target_rows, target_cols] #shape of the image is 500x500 pixels

In [3]:
#here is what is shown from the parameters
print(pc_params)

{'ellps': 'WGS84', 'a': 57.29577951308232, 'proj': 'eqc', 'lon_0': 0.0}


Load the .nc file and parse the radiance data

In [4]:
file = random.choice(os.listdir('../../GOES_Files/nc_files/'))
data = xr.open_dataset('../../GOES_Files/nc_files/' + file)
dat = data.metpy.parse_cf('Rad')
rad = dat.data

Calculate the brightness temperatures based on the radiance data and other information from the .nc file and save as an array

In [5]:
fk1 = float(data.metpy.parse_cf('planck_fk1'))
fk2 = float(data.metpy.parse_cf('planck_fk2'))
bc1 = float(data.metpy.parse_cf('planck_bc1'))
bc2 = float(data.metpy.parse_cf('planck_bc2'))
bt = (fk2/(xr.ufuncs.log(fk1/rad + 1)) - bc1)/bc2
bt = np.asarray(bt)

Define source and target areas <br />
Source area is from nc file <br />
Target Area is created using the previously defined parameters

In [6]:
source_area = geometry.AreaDefinition.from_cf(data)
target_area = geometry.AreaDefinition.from_extent('CA', pc_params, shape, target_extents)

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyproj/crs/crs.py:1216: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)


Resample the brightness temperatures with source and target areas and save as an array

In [7]:
result = grid.get_resampled_image(target_area, source_area, bt)
np.save('../../GOES_Files/npy_files/' + file[:-3], result)
data.close()
os.remove('../../GOES_Files/nc_files/' + file)

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyproj/crs/crs.py:1216: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)
